### Data Exploration and Processing

In [ ]:
!pip3 install pycocotools

In [ ]:
# Importing required libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import data_utils
import img_utils
from data_utils import get_transform, BloodCellDataset
from bcnet import get_bcnet
from train_eval_utils import (
    hocv_model, 
    kfcv_model, 
    plot_metrics_per_epoch, 
    train_model, 
    eval_model, 
    save_model,
    load_model,
    model_predict
)

DATA_PATH = "/storage/data"  # Change this to the relative path where your data is stored

Plotting sample image

In [ ]:
full_dataset = data_utils.BloodCellDataset(DATA_PATH)
image, target = full_dataset[0]
img_utils.plot_img_w_box(image, target)

The following preprocessing steps are performed here:

1. Reading in 'annotations.csv' and checking for duplicates, followed by dropping them.
2. Changing the class labels from 'rbc' and 'wbc' to 1 and 2 respectively. (0 is reserved for the background class)
3. Saving the new preprocessed data as a new .csv file called "clean_anno.csv".

In [ ]:
anno_df = pd.read_csv('{0}/annotations.csv'.format(DATA_PATH))
dup_df = data_utils.get_duplicates(anno_df)
to_drop=[dup_df.loc[(dup_df['image']=='image-1.png') & (dup_df['label']=='rbc'),:].index[0]]
to_drop.append(dup_df.loc[(dup_df['image']=='image-100.png') & (dup_df['label']=='wbc'),:].index[0])
to_drop.append(dup_df.loc[(dup_df['image']=='image-104.png') & (dup_df['label']=='wbc'),:].index[0])
to_drop.append(dup_df.loc[(dup_df['image']=='image-114.png') & (dup_df['label']=='wbc'),:].index[0])
clean_anno_df = anno_df.drop(to_drop)
clean_anno_df['label'] = np.where(clean_anno_df['label']=='rbc', 1, 2)
#clean_anno_df.to_csv('{0}/clean_anno.csv'.format(DATA_PATH), index=False)

Visualizing the distribution of RBCs to WBCs across all the images

In [ ]:
data_utils.get_class_distribution('{0}/clean_anno.csv'.format(DATA_PATH), ['image','label'])

### Cross Validation and Hyperparameter Tuning

In [ ]:
# 80-20 train-test split
train_indices, test_indices = train_test_split(range(100), test_size=0.2, random_state=42)

In [ ]:
# Hold-out cross validation (uncomment and run only if you wish to do this, otherwise use 5-fold CV in the next cell)

# frcnn = get_bcnet('retina', 3)
# hocv_model(
#     model=frcnn, 
#     dir_name='/storage/data', 
#     train_idxs=train_indices, 
#     val_idxs=test_indices, 
#     train_transforms=get_transform(True), 
#     val_transforms=get_transform(False), 
#     num_epochs=20, 
#     batch_size=2, 
#     learning_rate=0.01
#     )

In [ ]:
# Hyperparameter tuning here

model_to_tune = 'frcnn'  # Adjust to which model you wish to tune for. One of ('frcnn', 'retina', 'ssd')

train_hist, val_hist = kfcv_model(
    model_type=model_to_tune,
    dir_name=DATA_PATH,
    train_idxs=train_indices,
    train_transforms=get_transform(True),
    val_transforms=get_transform(False),
    num_epochs=30,   # Change this hyperparameter accordingly
    batch_size=2,   # Change this hyperparameter accordingly
    learning_rate=0.1,   # Change this hyperparameter accordingly
    k_folds=5
)

plot_metrics_per_epoch(train_hist, val_hist)

From our the validation scores of each model, we will use these hyperparameters for the final training of the three models:

* Faster R-CNN
    * Number of epochs: 15
    * Initial learning rate: 0.01
    * Batch size: 2

* RetinaNet
    * Number of epochs: 15
    * Initial learning rate: 0.05
    * Batch size: 2

* SSD
    * Number of epochs: 15
    * Initial learning rate: 0.005
    * Batch size: 2
    
   

### Training and evaluation

In [ ]:
# Train all three models

frcnn = get_bcnet(model_name='frcnn', num_classes=3, num_trainable_layers=1)
retinanet = get_bcnet(model_name='retina', num_classes=3, num_trainable_layers=1)
ssd_net = get_bcnet(model_name='ssd', num_classes=3, num_trainable_layers=1)

frcnn = train_model(
    model=frcnn,
    dir_name=DATA_PATH,
    train_idxs=train_indices,
    transforms=get_transform(train=True),
    num_epochs=15,
    batch_size=2,
    learning_rate=0.1
)

retinanet = train_model(
    model=retinanet,
    dir_name=DATA_PATH,
    train_idxs=train_indices,
    transforms=get_transform(train=True),
    num_epochs=15,
    batch_size=2,
    learning_rate=0.05
)

ssd_net = train_model(
    model=ssd_net,
    dir_name=DATA_PATH,
    train_idxs=train_indices,
    transforms=get_transform(train=True),
    num_epochs=15,
    batch_size=2,
    learning_rate=0.005
)

In [ ]:
# Evaluate models on test set

frcnn_test = eval_model(
    model=frcnn,
    dir_name=DATA_PATH,
    test_idxs=test_indices,
    test_transforms=get_transform(train=False)
)

retina_test = eval_model(
    model=retinanet,
    dir_name=DATA_PATH,
    test_idxs=test_indices,
    test_transforms=get_transform(train=False)
)

ssd_test = eval_model(
    model=ssd_net,
    dir_name=DATA_PATH,
    test_idxs=test_indices,
    test_transforms=get_transform(train=False)
)

for res in zip(['Faster R-CNN', 'RetinaNet', 'SSD'], [frcnn_test, retina_test, ssd_test]):
    print(
        '{0} --> COCO-mAP: {1:.4f}, mAP-50: {2:.4f}, mAP-75: {3:.4f}'.format(
            res[0], res[1]['coco_map'], res[1]['map_50'], res[1]['map_75']
            )
        )

### Generate predictions from final models

Take note that we use non-maximum suppression here to remove "excess" bounding boxes. 

In [ ]:
test_dataset = BloodCellDataset(DATA_PATH, get_transform(train=False))
image, actual = test_dataset[0]
predicted = model_predict(frcnn_test, image)
# Plot the predicted bounding boxes
img_utils.plot_img_w_box(image, predicted, nms=True, iou_threshold=0.4)

In [ ]:
# Plot the actual bounding boxes
img_utils.plot_img_w_box(image, actual)

In [ ]:
# Saving models

save_model(model=frcnn, save_path="saved_models/best_frcnn.pth")
save_model(model=retinanet, save_path="saved_models/best_retina.pth")
save_model(model=ssd_net, save_path="saved_models/best_ssd.pth")

In [ ]:
# Example on how to load a model from saved path
new_frcnn = get_bcnet('frcnn', 3, 1)
load_model(new_frcnn, "saved_models/best_frcnn.pth")